<a href="https://colab.research.google.com/github/essiesalari/Australian-Celebrities-Face-Recognition-VGG16/blob/main/Australian_Celebrities_Face_Recognition_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Australian Celebrities Face Recognition-VGG16**

## Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Import datatset

In [2]:
! pip install bing_image_downloader

In [3]:
from bing_image_downloader import downloader

In [ ]:
# Query
search_queries=[
    'Nicole Kidman',
    'Mel Gibson',
    'Liam Hemsworth',
    'Cathy Freeman',
    'Margot Robbie'
]

dataset_path = 'dataset'

# Imgae downloder function
def donwloader(query, n_sample, dataset_name):
  downloader.download(query, limit=n_sample, output_dir=os.path.join(dataset_path, dataset_name))

for query in search_queries:
  donwloader(query, n_sample=30, dataset_name='train')
  donwloader(query, n_sample=5, dataset_name='val')
  donwloader(query, n_sample=3, dataset_name='test')

In [5]:
dataset_path='/content/dataset'
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
test_path = os.path.join(dataset_path, 'test')

In [6]:
nr_files = nr_train_files = nr_val_files = nr_test_files = 0

for root, dirc, files in os.walk(dataset_path):
  nr_files += len(files)
print("#files: ", nr_files)
for root, dirc, files in os.walk(train_path):
  nr_train_files += len(files)
print("#train_files: ", nr_train_files)
for root, dirc, files in os.walk(val_path):
  nr_val_files += len(files)
print("#val_files: ", nr_val_files)
for root, dirc, files in os.walk(test_path):
  nr_test_files += len(files)
print("#test_files: ", nr_test_files)

#files:  144
#train_files:  110
#val_files:  23
#test_files:  11


# Build  a New Model

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [20]:
batch_size = 128
lr = 0.001
epoch = 100

# Define Image data generator
train_datagen = ImageDataGenerator(
                                  rescale=1./255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen =  ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 110 images belonging to 5 classes.
Found 23 images belonging to 5 classes.
Found 11 images belonging to 5 classes.


## VGG16

In [25]:
# Load VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tunig
for layer in base_model.layers[:-4]:
  layer.trainable = False

# Add global average layer and dropout for regularization
net = base_model.output
net = GlobalAveragePooling2D()(net)
net = Dropout(0.6)(net)

# Creat the new model with VGG16 (feature selection) + SVC (classifier)
new_model = Model(inputs=base_model.input, outputs=net)

train_features = new_model.predict(train_generator)
train_labels = train_generator.classes

val_features = new_model.predict(val_generator)
val_labels = val_generator.classes

# Train SVM Classifier
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels )

1/1 [==============================] - 0s 495ms/step


SVC(kernel='linear')

## Train and Evaluate the New_Model

In [26]:
# Evaluate on validation set
val_accuracy = svm.score(val_features, val_labels)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.21739130434782608


## ResNet50

In [28]:
# Load Resnet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tunig
for layer in base_model.layers[:-4]:
  layer.trainable = False

# Add global average layer and dropout for regularization
net = base_model.output
net = GlobalAveragePooling2D()(net)
net = Dropout(0.6)(net)

# Creat the new model with VGG16 (feature selection) + SVC (classifier)
new_model = Model(inputs=base_model.input, outputs=net)

train_features = new_model.predict(train_generator)
train_labels = train_generator.classes

val_features = new_model.predict(val_generator)
val_labels = val_generator.classes

# Train SVM Classifier
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels )

1/1 [==============================] - 2s 2s/step


SVC(kernel='linear')

In [29]:
# Evaluate on validation set
val_accuracy = svm.score(val_features, val_labels)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.17391304347826086


## InceptionV3

In [31]:
# Load Resnet50
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tunig
for layer in base_model.layers[:-4]:
  layer.trainable = False

# Add global average layer and dropout for regularization
net = base_model.output
net = GlobalAveragePooling2D()(net)
net = Dropout(0.6)(net)

# Creat the new model with VGG16 (feature selection) + SVC (classifier)
new_model = Model(inputs=base_model.input, outputs=net)

train_features = new_model.predict(train_generator)
train_labels = train_generator.classes

val_features = new_model.predict(val_generator)
val_labels = val_generator.classes

# Train SVM Classifier
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels )

1/1 [==============================] - 3s 3s/step


SVC(kernel='linear')

In [32]:
# Evaluate on validation set
val_accuracy = svm.score(val_features, val_labels)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.043478260869565216


## EfficientNetB0

In [35]:
# Load Resnet50
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tunig
for layer in base_model.layers[:-4]:
  layer.trainable = False

# Add global average layer and dropout for regularization
net = base_model.output
net = GlobalAveragePooling2D()(net)
net = Dropout(0.6)(net)

# Creat the new model with VGG16 (feature selection) + SVC (classifier)
new_model = Model(inputs=base_model.input, outputs=net)

train_features = new_model.predict(train_generator)
train_labels = train_generator.classes

val_features = new_model.predict(val_generator)
val_labels = val_generator.classes

# Train SVM Classifier
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels )

1/1 [==============================] - 2s 2s/step


SVC(kernel='linear')

In [36]:
# Evaluate on validation set
val_accuracy = svm.score(val_features, val_labels)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.2608695652173913
